# Практическая работа 7

## 1 Найти данные для задачи классификации или для задачи регрессии

In [2]:
import os
import pandas as pd

PATH = 'D:/projects/PycharmProjects/MireaBigData/pr4/'
file_path = os.path.join(PATH, 'smartphones.csv')

data = pd.read_csv(file_path)
data.info()

# Числовые признаки
num_cols = [
    'avg_rating',
    '5G_or_not',
    'num_cores',
    'processor_speed',
    'battery_capacity',
    'fast_charging_available',
    'fast_charging',
    'ram_capacity',
    'internal_memory',
    'screen_size',
    'refresh_rate',
    'num_rear_cameras',
    'primary_camera_rear',
    'primary_camera_front',
    'extended_memory_available',
    'resolution_height',
    'resolution_width',
]

# Категориальные признаки
cat_cols = [
    'brand_name',
    'model',
    'os',
]

feature_cols = num_cols + cat_cols
target_col = 'price'

data.dropna(inplace=True)
y = data[target_col]
data.drop(target_col, axis=1, inplace=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 941 entries, 0 to 940
Data columns (total 22 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   brand_name                 941 non-null    object 
 1   model                      941 non-null    object 
 2   price                      941 non-null    int64  
 3   avg_rating                 842 non-null    float64
 4   5G_or_not                  941 non-null    int64  
 5   processor_brand            922 non-null    object 
 6   num_cores                  935 non-null    float64
 7   processor_speed            899 non-null    float64
 8   battery_capacity           930 non-null    float64
 9   fast_charging_available    941 non-null    int64  
 10  fast_charging              741 non-null    float64
 11  ram_capacity               941 non-null    int64  
 12  internal_memory            941 non-null    int64  
 13  screen_size                941 non-null    float64

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, ColumnTransformer

data.replace(" ", np.nan, inplace=True)
data.columns = [str(t) for t in data.columns]
X_train, X_valid, y_train, y_valid = train_test_split(data, y, train_size=0.8, random_state=42)

# отнормируем числовые признаки и закодируем категориальные
preprocessor = make_column_transformer(
    (make_pipeline(StandardScaler(), RobustScaler()), num_cols),
    (OneHotEncoder(), cat_cols)
)

## 2 Реализовать баггинг

In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline

random_forest = RandomForestRegressor()

# Сетка параметров для поиска
params_grid = {
    "max_depth": [9, 15, 18, 24],
    "min_samples_leaf": [1, 2, 3],
    "min_samples_split": [8, 10, 12]
}

# Измените scoring на метрику для регрессии
grid_search_random_forest = GridSearchCV(
    estimator=random_forest,
    param_grid=params_grid,
    scoring="neg_mean_squared_error",  # Изменено
    cv=5  # Можно добавить кросс-валидацию
)

# Создаем пайплайн
model = make_pipeline(
    preprocessor,
    grid_search_random_forest
)

# Обучение модели
import time
start = time.time()
model.fit(X_train, y_train)
end = time.time()
print("Время обучения:", end - start)

# Выводим наилучшие параметры
print("Наилучшие параметры:", grid_search_random_forest.best_params_)

Время обучения: 57.77694010734558
Наилучшие параметры: {'max_depth': 18, 'min_samples_leaf': 1, 'min_samples_split': 8}


In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# Предсказания модели
y_pred = model.predict(X_train)

# Вычисление RMSE
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
print("Среднеквадратичное отклонение (RMSE):", rmse)
# Вычисление MAE
mae = mean_absolute_error(y_train, y_pred)
print("Среднее абсолютное отклонение (MAE):", mae)

Среднеквадратичное отклонение (RMSE): 9012.831434917574
Среднее абсолютное отклонение (MAE): 2461.299210986373


## 3 Реализовать бустинг на тех же данных, что использовались для баггинга

In [6]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

# Создаем модель бустинга
gradient_boosting = GradientBoostingRegressor()

# Определяем сетку параметров для поиска лучших гиперпараметров
params_grid = {
    "n_estimators": [100, 200],
    "max_depth": [3, 4, 5],
    "learning_rate": [0.01, 0.05, 0.1],
    "min_samples_split": [2, 3, 4],
    "min_samples_leaf": [1, 2, 3]
}

# Настраиваем Grid Search
grid_search_gradient_boosting = GridSearchCV(
    estimator=gradient_boosting,
    param_grid=params_grid,
    scoring="neg_mean_squared_error",
    cv=5
)

# Создаем пайплайн
model = make_pipeline(
    preprocessor,
    grid_search_gradient_boosting
)

# Обучение модели
import time
start = time.time()
model.fit(X_train, y_train)
end = time.time()
print("Время обучения (Gradient Boosting):", end - start)

# Выводим наилучшие параметры
print("Наилучшие параметры:", grid_search_gradient_boosting.best_params_)

Время обучения (Gradient Boosting): 209.70157742500305
Наилучшие параметры: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 3, 'n_estimators': 200}


In [7]:
# Оценка модели
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# Предсказания модели
y_pred = model.predict(X_train)

# Вычисление MAE и RMSE
mae = mean_absolute_error(y_train, y_pred)
rmse = np.sqrt(mean_squared_error(y_train, y_pred))

print("Среднее абсолютное отклонение (MAE):", mae)
print("Среднеквадратичное отклонение (RMSE):", rmse)

Среднее абсолютное отклонение (MAE): 1497.8710222309887
Среднеквадратичное отклонение (RMSE): 1716.3618951263627


## 4 Сравнить результаты работы алгоритмов (время работы и качество моделей). Сделать выводы